In [24]:
from utils.data_utils import *
from model.fl_model import VerticalFLModel
from model.simple_fl_model import PCAVerticalFLModel
from model.single_party_model import SingleParty
from model.split_nn_model import SplitNNModel
from utils.split_train_test import split_train_test

from torch.utils.tensorboard import SummaryWriter

import os.path
import wget
import bz2
import shutil
import numpy as np

import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score
from datetime import datetime


In [4]:
!pip install torch_optimizer

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 61 kB 1.9 MB/s eta 0:00:01


In [6]:
if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/phishing"):
    print("Downloading phishing data")
    wget.download("https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/phishing",
                  "data/phishing")

if not os.path.isfile("data/phishing.train") or not os.path.isfile("data/phishing.test"):
    split_train_test("phishing", file_type='libsvm', test_rate=0.1)

In [26]:
num_parties = 2
x_scaler_wrapper = []
y_scaler_wrapper = []
x_normalizer_wrapper = []

cross_valid_data = load_data_cross_validation("phishing.train", num_parties=num_parties,
                                              file_type='libsvm', n_fold=5, use_cache=False,
                                              x_scaler_wrapper=x_scaler_wrapper, y_scaler_wrapper=y_scaler_wrapper,
                                              x_normalizer_wrapper=x_normalizer_wrapper)

Loading data from file
Percentage of positive labels: 55.59352698763694
Scaling labels
5 fold spliltting
Fold 0 finished
Fold 1 finished
Fold 2 finished
Fold 3 finished
Fold 4 finished


In [36]:
cross_valid_data[3][0][0].shape

(7959, 34)

In [25]:
xs_train, y_train, xs_test, y_test, x_train, x_test = load_data_train_test("phishing", num_parties=1,
                                                                           test_size=0.2, file_type='libsvm')

Data splitting


TypeError: np.matrix is not supported. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html

In [21]:
cross_valid_data[0][0][1].shape

IndexError: list index out of range

In [15]:
# SecureBoost with DP (use XGBoost instead since SecureBoost is lossless)
num_parties = 2
x_scaler_wrapper = []
y_scaler_wrapper = []
x_normalizer_wrapper = []
cross_valid_data = load_data_cross_validation("phishing.train", num_parties=num_parties,
                                              file_type='libsvm', n_fold=5, use_cache=False,
                                              x_scaler_wrapper=x_scaler_wrapper, y_scaler_wrapper=y_scaler_wrapper,
                                              x_normalizer_wrapper=x_normalizer_wrapper)
xs_test, y_test = load_data_cross_validation("phishing.test", num_parties=num_parties,
                                             file_type='libsvm', n_fold=1, use_cache=False,
                                             x_scaler_wrapper=x_scaler_wrapper, y_scaler_wrapper=y_scaler_wrapper,
                                             x_normalizer_wrapper=x_normalizer_wrapper)[0]

Loading data from file
Percentage of positive labels: 55.59352698763694
Scaling labels
5 fold spliltting
Fold 0 finished
Fold 1 finished
Fold 2 finished
Fold 3 finished
Fold 4 finished
Loading data from file
Percentage of positive labels: 56.6003616636528
Scaling labels
1 fold spliltting


array([1., 0., 0., ..., 0., 1., 0.])